# Environment Test Setup

## Imports

In [1]:
from kaggle_environments import evaluate, make, utils
import numpy as np

## Test Setup

In [ ]:
env = make("connectx", debug=True)
env.render()

In [ ]:
# 1. Create the environment
env = make("connectx", debug=True)

# 2. You must 'run' or 'reset' the environment for a board to exist
env.reset()

# 3. Explicitly print the human-readable render
print(env.render(mode="ansi"))

# Agent Definitions

## Random Agent

In [2]:
def random_agent(observation, configuration):
    """
    A simple agent that plays a random, but valid, move.
    """
    import random
    # observation.board is a 1D list representing the grid.
    # 0 = empty, 1 = player 1, 2 = player 2.
    # The top row corresponds to indices 0 through (configuration.columns - 1).
    
    # Find all columns that are not completely full
    valid_moves = [
        col for col in range(configuration.columns) 
        if observation.board[col] == 0
    ]
    
    # Return a random choice among the valid columns
    return random.choice(valid_moves)

## Leftmost Agent

In [3]:
def leftmost_agent(observation, configuration):
    """Always picks the leftmost valid column."""
    valid_moves = [col for col in range(configuration.columns) if observation.board[col] == 0]
    return valid_moves[0]

## Reactive Agent
1. Level 1: The One-Move Lookahead (Reactive)
The Goal: Build an agent that isn't blind. It should see a winning move and take it, and see an opponent’s winning move and block it.

Concepts: Simulating moves, identifying winning states, and basic heuristics.

Outcome: A "Smart-ish" agent that beats Random and Leftmost 100% of the time.

In [10]:
def reactive_agent(observation, configuration):
    """
    A Level 1 AI that identifies immediate wins and blocks immediate threats.
    All helper functions are defined inside or must be available in scope.
    
    Args:
        observation: Kaggle object (board, mark).
        configuration: Kaggle object (rows, columns, inarow).
        
    Returns:
        int: The column index to play.
    """
    import numpy as np
    import random

    # --- HELPER FUNCTIONS (Must be inside or accessible) ---
    def local_drop_piece(grid, col, piece, config):
        next_grid = grid.copy()
        for r in range(config.rows - 1, -1, -1):
            if next_grid[r][col] == 0:
                next_grid[r][col] = piece
                return next_grid
        return next_grid

    def local_is_win(grid, piece, config):
        # Horizontal
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                window = list(grid[r, c:c+config.inarow])
                if window.count(piece) == config.inarow: return True
        # Vertical
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow - 1)):
                window = list(grid[r:r+config.inarow, c])
                if window.count(piece) == config.inarow: return True
        # Positive Diagonal
        for r in range(config.rows - (config.inarow - 1)):
            for c in range(config.columns - (config.inarow - 1)):
                window = [grid[r+i, c+i] for i in range(config.inarow)]
                if window.count(piece) == config.inarow: return True
        # Negative Diagonal
        for r in range(config.inarow - 1, config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                window = [grid[r-i, c+i] for i in range(config.inarow)]
                if window.count(piece) == config.inarow: return True
        return False

    # --- AGENT LOGIC ---
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    me = observation.mark
    enemy = 1 if me == 2 else 2
    
    valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]

    # 1. Check for Win
    for col in valid_moves:
        potential_grid = local_drop_piece(grid, col, me, configuration)
        if local_is_win(potential_grid, me, configuration):
            return int(col) # Force standard Python int

    # 2. Check for Block
    for col in valid_moves:
        potential_grid = local_drop_piece(grid, col, enemy, configuration)
        if local_is_win(potential_grid, enemy, configuration):
            return int(col) # Force standard Python int

    # 3. Random fallback
    return int(random.choice(valid_moves))

In [ ]:
# For unit test purpose
def drop_piece(grid, col, piece, config):
    """
    Simulates dropping a piece into a specific column of the board.

    Args:
        grid (np.ndarray): A 2D numpy array representing the current board state.
        col (int): The index of the column (0 to config.columns-1) where the piece is dropped.
        piece (int): The player's mark (1 or 2).
        config (struct): The game configuration containing board dimensions (rows, columns).

    Returns:
        np.ndarray: A new 2D numpy array representing the board AFTER the move.
                   Returns the original grid if the move is invalid (column full).
    """
    # Create a deep copy so we don't modify the original 'real' board
    next_grid = grid.copy()
    
    # We loop from the bottom row up to the top row
    # range(start, stop, step) -> (bottom_index, -1, go_up_by_one)
    for row in range(config.rows - 1, -1, -1):
        if next_grid[row][col] == 0:
            next_grid[row][col] = piece
            return next_grid
            
    # If we get here, the column was full
    return next_grid

In [ ]:
# for unit test purpose
def is_win(grid, piece, config):
    """
    Scans the board to determine if the specified player has won.

    Args:
        grid (np.ndarray): The 2D numpy array representing the board.
        piece (int): The player mark to check for (1 or 2).
        config (struct): Configuration containing 'rows', 'columns', and 'inarow'.

    Returns:
        bool: True if the player has 'config.inarow' in a line, False otherwise.
    """
    # 1. Check Horizontal
    for r in range(config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = list(grid[r, c:c+config.inarow])
            if window.count(piece) == config.inarow:
                return True

    # 2. Check Vertical
    for c in range(config.columns):
        for r in range(config.rows - (config.inarow - 1)):
            window = list(grid[r:r+config.inarow, c])
            if window.count(piece) == config.inarow:
                return True

    # 3. Check Positive Diagonal (/)
    for r in range(config.rows - (config.inarow - 1)):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r+i, c+i] for i in range(config.inarow)]
            if window.count(piece) == config.inarow:
                return True

    # 4. Check Negative Diagonal (\)
    for r in range(config.inarow - 1, config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r-i, c+i] for i in range(config.inarow)]
            if window.count(piece) == config.inarow:
                return True

    return False

### Unit Tests

In [ ]:
# Test drop_piece()
class MockConfig:
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns

def test_drop_piece_advanced():
    config_2x2 = MockConfig(rows=2, columns=2)
    board = np.zeros((2, 2), dtype=int)
    print("Step 0: Empty 2x2 Board\n", board)
    
    # 1) Drop piece 1 in column 0
    board = drop_piece(board, col=0, piece=1, config=config_2x2)
    print("\nStep 1: P1 in Col 0\n", board)
    assert board[1][0] == 1
    
    # 2) Drop piece 2 in column 1
    board = drop_piece(board, col=1, piece=2, config=config_2x2)
    print("\nStep 2: P2 in Col 1\n", board)
    assert board[1][1] == 2
    
    # 3) Drop piece 1 in column 0
    board = drop_piece(board, col=0, piece=1, config=config_2x2)
    print("\nStep 3: P1 in Col 0 (should stack)\n", board)
    assert board[0][0] == 1
    
    # 4) Drop piece 2 in column 0 (Wait, this column is already full!)
    # Since it's a 2x2 board, Column 0 now has two pieces. 
    # Let's see how our function handles a 3rd piece in a 2-high column.
    board_after_full = drop_piece(board, col=0, piece=2, config=config_2x2)
    print("\nStep 4: P2 in FULL Col 0 (should not change board)\n", board_after_full)
    
    # Verification
    assert np.array_equal(board, board_after_full), "Error: Board changed even though column was full!"
    print("\n✅ All tests passed! The simulator correctly handles stacking and full columns.")

test_drop_piece_advanced()

In [ ]:
# Test is_win()
class MockConfig:
    def __init__(self, rows, columns, inarow):
        self.rows = rows
        self.columns = columns
        self.inarow = inarow

def test_is_win():
    # Setup: 3x3 board where you only need 2-in-a-row to win
    config_3x3 = MockConfig(rows=3, columns=3, inarow=2)
    
    # Test 1: Horizontal Win
    grid_h = np.array([
        [0, 0, 0],
        [0, 0, 0],
        [1, 1, 0]  # Two 1s in the bottom row
    ])
    assert is_win(grid_h, 1, config_3x3) == True
    assert is_win(grid_h, 2, config_3x3) == False # Player 2 hasn't won
    print("Test 1: Horizontal Passed")

    # Test 2: Vertical Win
    grid_v = np.array([
        [2, 0, 0],
        [2, 0, 0],
        [1, 0, 0]
    ])
    assert is_win(grid_v, 2, config_3x3) == True
    print("Test 2: Vertical Passed")

    # Test 3: Positive Diagonal (/)
    grid_pd = np.array([
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 0]
    ])
    assert is_win(grid_pd, 1, config_3x3) == True
    print("Test 3: Positive Diagonal Passed")

    # Test 4: Negative Diagonal (\)
    grid_nd = np.array([
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 0]
    ])
    assert is_win(grid_nd, 2, config_3x3) == True
    print("Test 4: Negative Diagonal Passed")

    # Test 5: No Win
    grid_none = np.array([
        [1, 2, 1],
        [2, 1, 2],
        [1, 2, 1]
    ])
    # Even though it's full, nobody has 2-in-a-row? 
    # Wait, in a 3x3 with 2-in-a-row, almost any move wins.
    # Let's use an empty board.
    grid_empty = np.zeros((3,3))
    assert is_win(grid_empty, 1, config_3x3) == False
    print("Test 5: Empty Board Passed")

    print("\n✅ All Vision Tests Passed! The agent can now see.")

test_is_win()

In [ ]:
class MockObject:
    """A simple class to turn a dictionary into an object with dot-notation access."""
    def __init__(self, **entries):
        self.__dict__.update(entries)

def test_reactive_agent_blocking():
    """
    Scenario: The enemy (Player 2) has 3-in-a-row horizontally on the bottom row.
    The reactive_agent (Player 1) MUST block the 4th spot.
    
    Args:
        None
        
    Returns:
        None: Prints success or raises an AssertionError.
    """
    # 1. Setup Configuration using our MockObject
    config = MockObject(rows=6, columns=7, inarow=4)
    
    # 2. Construct the Board State
    # Bottom row (index 5): [2, 2, 2, 0, 0, 0, 0]
    grid = np.zeros((6, 7), dtype=int)
    grid[5][0] = 2
    grid[5][1] = 2
    grid[5][2] = 2
    
    # 3. Create the Observation using our MockObject
    # This ensures observation.board and observation.mark work correctly!
    obs = MockObject(board=list(grid.flatten()), mark=1)
    
    # 4. Execute Agent Logic
    chosen_col = reactive_agent(obs, config)
    
    # 5. Verification
    print(f"Enemy has 3-in-a-row in cols 0,1,2. Agent chose: {chosen_col}")
    assert chosen_col == 3, f"Fail: Agent should have blocked col 3, but chose {chosen_col}"
    print("✅ Blocking Scenario Passed!")

def test_reactive_agent_winning():
    """
    Scenario: The agent (Player 1) has 3 pieces stacked vertically in Column 4.
    The agent MUST recognize that playing in Column 4 results in an immediate win.
    
    Args:
        None (Uses the MockObject class and existing reactive_agent).
        
    Returns:
        None: Prints success or raises an AssertionError.
    """
    # 1. Setup Configuration using our MockObject
    config = MockObject(rows=6, columns=7, inarow=4)
    
    # 2. Construct the Board State
    # Three '1's in column 4, starting from the bottom.
    grid = np.zeros((6, 7), dtype=int)
    grid[5][4] = 1 
    grid[4][4] = 1 
    grid[3][4] = 1 
    
    # 3. Create the Observation using our MockObject
    obs = MockObject(board=list(grid.flatten()), mark=1)
    
    # 4. Execute Agent Logic
    chosen_col = reactive_agent(obs, config)
    
    # 5. Verification
    print(f"Agent (P1) sees 3-in-a-row in Col 4. Agent chose: {chosen_col}")
    assert chosen_col == 4, f"Fail: Agent should have won in Col 4, but chose {chosen_col}"
    print("✅ Winning Scenario Passed!")

# Run the test
test_reactive_agent_blocking()
test_reactive_agent_winning()

## Strategic Agent (MinMax)

Level 2: The Minimax Tree (Strategic)The Goal: Teach the agent to think ahead $N$ moves. It will assume the opponent is also playing optimally and choose the path that leads to the best guaranteed outcome.
Concepts: Recursion, the "Min" and "Max" players, and Depth-Limited Search.
Outcome: An agent that can compete with the built-in negamax.

### Agent Code

In [4]:
def minimax_agent(observation, configuration):
    """
    A Level 2 Strategic AI using Depth-Limited Minimax.
    Contains all necessary logic to be fully self-contained for Kaggle.
    """
    import numpy as np
    import random

    # --- SETTINGS ---
    DEPTH = 3 # Lookahead depth
    
    # --- HELPER: DROP PIECE ---
    def local_drop_piece(grid, col, piece, config):
        next_grid = grid.copy()
        for r in range(config.rows - 1, -1, -1):
            if next_grid[r][col] == 0:
                next_grid[r][col] = piece
                return next_grid
        return next_grid

    # --- HELPER: IS WIN ---
    def local_is_win(grid, piece, config):
        # Horizontal
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r, c+i] == piece for i in range(config.inarow)): return True
        # Vertical
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow - 1)):
                if all(grid[r+i, c] == piece for i in range(config.inarow)): return True
        # Diagonals
        for r in range(config.rows - (config.inarow - 1)):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r+i, c+i] == piece for i in range(config.inarow)): return True
        for r in range(config.inarow - 1, config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r-i, c+i] == piece for i in range(config.inarow)): return True
        return False

    # --- HELPER: HEURISTIC SCORER ---
    def local_get_score(grid, piece, config):
        score = 0
        enemy = 1 if piece == 2 else 2
        
        def evaluate_window(window, p, e):
            w_score = 0
            if window.count(p) == 4: w_score += 1000000
            elif window.count(p) == 3 and window.count(0) == 1: w_score += 100
            elif window.count(p) == 2 and window.count(0) == 2: w_score += 10
            if window.count(e) == 3 and window.count(0) == 1: w_score -= 10000
            if window.count(e) == 4: w_score -= 1000000
            return w_score

        # Scan Horizontal, Vertical, Diagonals (Simplified for brevity)
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window(list(grid[r, c:c+config.inarow]), piece, enemy)
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow-1)):
                score += evaluate_window(list(grid[r:r+config.inarow, c]), piece, enemy)
        for r in range(config.rows - (config.inarow-1)):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r+i, c+i] for i in range(config.inarow)], piece, enemy)
        for r in range(config.inarow-1, config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r-i, c+i] for i in range(config.inarow)], piece, enemy)
        return score

    # --- CORE: RECURSIVE MINIMAX ---
    def local_minimax(grid, depth, is_maximizing, piece, config):
        enemy = 1 if piece == 2 else 2
        
        if local_is_win(grid, piece, config): return 1000000
        if local_is_win(grid, enemy, config): return -1000000
        if depth == 0: return local_get_score(grid, piece, config)
        
        valid_moves = [c for c in range(config.columns) if grid[0][c] == 0]
        if not valid_moves: return 0

        if is_maximizing:
            best_score = -float('inf')
            for col in valid_moves:
                temp_grid = local_drop_piece(grid, col, piece, config)
                score = local_minimax(temp_grid, depth - 1, False, piece, config)
                best_score = max(score, best_score)
            return best_score
        else:
            best_score = float('inf')
            for col in valid_moves:
                temp_grid = local_drop_piece(grid, col, enemy, config)
                score = local_minimax(temp_grid, depth - 1, True, piece, config)
                best_score = min(score, best_score)
            return best_score

    # --- AGENT EXECUTION ---
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    me = observation.mark
    valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]
    
    best_score = -float('inf')
    best_move = random.choice(valid_moves)
    
    for col in valid_moves:
        temp_grid = local_drop_piece(grid, col, me, configuration)
        score = local_minimax(temp_grid, DEPTH-1, False, me, configuration)
        if score > best_score:
            best_score = score
            best_move = col
            
    return int(best_move)

### Unit Tests

In [ ]:
class MockObject:
    """A simple class to turn a dictionary into an object with dot-notation access."""
    def __init__(self, **entries):
        self.__dict__.update(entries)

def get_score(grid, piece, config):
    """
    Evaluates the 'goodness' of a board state for a specific player.
    Note: 'Double counting' overlapping windows is a feature that rewards 
    centrality and connectivity.
    """
    enemy = 1 if piece == 2 else 2
    score = 0
    
    def evaluate_window(window, piece, config):
        window_score = 0
        enemy = 1 if piece == 2 else 2
        
        # --- MY SCORES ---
        if window.count(piece) == 4:
            window_score += 1000000   # Victory
        elif window.count(piece) == 3 and window.count(0) == 1:
            window_score += 100       # Strong potential
        elif window.count(piece) == 2 and window.count(0) == 2:
            window_score += 10        # Building blocks
            
        # --- ENEMY SCORES ---
        if window.count(enemy) == 4:
            window_score -= 1000000   # Immediate Loss (Hard Floor)
        elif window.count(enemy) == 3 and window.count(0) == 1:
            window_score -= 10000     # Critical Threat (Must Block)
            
        return window_score

    # 1. Horizontal Score
    for r in range(config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = list(grid[r, c:c+config.inarow])
            score += evaluate_window(window, piece, config)

    # 2. Vertical Score
    for c in range(config.columns):
        for r in range(config.rows - (config.inarow - 1)):
            window = list(grid[r:r+config.inarow, c])
            score += evaluate_window(window, piece, config)

    # 3. Positive Diagonal (/) Score
    for r in range(config.rows - (config.inarow - 1)):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r+i, c+i] for i in range(config.inarow)]
            score += evaluate_window(window, piece, config)

    # 4. Negative Diagonal (\) Score
    for r in range(config.inarow - 1, config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r-i, c+i] for i in range(config.inarow)]
            score += evaluate_window(window, piece, config)

    return score

def test_heuristic_scoring_full():
    """
    Checks the scoring hierarchy: 
    Winning > Blocking Enemy > Having 3-in-a-row > Having 2-in-a-row.
    """
    config = MockObject(rows=6, columns=7, inarow=4)
    
    # 1. My 2-in-a-row
    grid_2 = np.zeros((6, 7), dtype=int)
    grid_2[5, 0:2] = 1
    score_2 = get_score(grid_2, 1, config)
    
    # 2. My 3-in-a-row
    grid_3 = np.zeros((6, 7), dtype=int)
    grid_3[5, 0:3] = 1
    score_3 = get_score(grid_3, 1, config)
    
    # 3. Enemy's 3-in-a-row (Danger!)
    grid_enemy_3 = np.zeros((6, 7), dtype=int)
    grid_enemy_3[5, 0:3] = 2 
    score_enemy_3 = get_score(grid_enemy_3, 1, config)
    
    # 4. My 4-in-a-row (Victory)
    grid_4 = np.zeros((6, 7), dtype=int)
    grid_4[5, 0:4] = 1
    score_4 = get_score(grid_4, 1, config)
    
    # 5. Enemy's 4-in-a-row (Loss)
    grid_enemy_4 = np.zeros((6, 7), dtype=int)
    grid_enemy_4[5, 0:4] = 2
    score_enemy_4 = get_score(grid_enemy_4, 1, config)

    print(f"--- Heuristic Scoring Results ---")
    print(f"My 4-in-a-row:     {score_4}")
    print(f"My 3-in-a-row:     {score_3}")
    print(f"My 2-in-a-row:     {score_2}")
    print(f"Enemy 3-in-a-row:  {score_enemy_3}")
    print(f"Enemy 4-in-a-row:  {score_enemy_4}")

    # Hierarchy Assertions
    assert score_4 > score_3 > score_2
    assert score_enemy_3 < 0
    assert score_enemy_4 < score_enemy_3
    print("\n✅ All Heuristic Hierarchy Tests Passed!")

# Run the test
test_heuristic_scoring_full()

## Optimized Agent (Alpha Beta Pruning)

3. Level 3: Alpha-Beta Pruning (Optimized)
The Goal: Make the search faster. Right now, Minimax looks at every single branch. Pruning allows us to ignore branches that are obviously worse, letting us look 6–8 moves ahead instead of just 3.

Concepts: Branch cutting and search efficiency.

Outcome: A competitive agent ready for the Kaggle leaderboard.

# Arena Play
Play a game between two agents

In [ ]:
from kaggle_environments import make
env = make("connectx", debug=True)
env.run([random_agent, "negamax"])
env.render(mode="ipython", width=500, height=450)

Play Your Agent

In [ ]:
# "None" represents which agent you'll manually play as (first or second player).
from kaggle_environments import make
env = make("connectx", debug=True)
env.play([None, "negamax"], width=500, height=450)

# Evaluation
Evaluate the agent in batch test runs against benchmark agents

In [8]:
def agent_battle(agent1, agent2, n_rounds=100):
    # Get names for printing
    name1 = agent1.__name__ if hasattr(agent1, '__name__') else str(agent1)
    name2 = agent2.__name__ if hasattr(agent2, '__name__') else str(agent2)
    
    config = {'rows': 6, 'columns': 7, 'inarow': 4}

    # Run the rounds
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds // 2)
    outcomes += [[b, a] for [a, b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds - n_rounds // 2)]
    # Calculate stats based on corrected rewards
    # outcomes.count([1, 0]) = Agent 1 wins
    # outcomes.count([0.5, 0.5]) = Ties
    # outcomes.count([None, 1]) = Agent 1 crashed
    
    a1_wins = outcomes.count([1, 0])
    a2_wins = outcomes.count([0, 1])
    ties = outcomes.count([0.5, 0.5])
    invalid = len(outcomes) - (a1_wins + a2_wins + ties)
    
    print(f"--- Results: {name1} vs {name2} ---")
    print(f"{name1} Win Rate: {np.round(a1_wins / len(outcomes), 2)}")
    print(f"{name2} Win Rate: {np.round(a2_wins / len(outcomes), 2)}")
    print(f"Ties: {np.round(ties / len(outcomes), 2)}")
    print(f"Invalid Plays (Crashes): {np.round(invalid / len(outcomes), 2)}")

## Random vs Negamax

In [ ]:
n_rounds = 10
print("Random vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle(random_agent, "negamax", n_rounds)

## Leftmost vs Random

In [ ]:
n_rounds = 100
print("Random vs Leftmost")
print(str(n_rounds) + " rounds...")
agent_battle(random_agent, leftmost_agent, n_rounds)

## Negamax vs Negamax

In [ ]:
n_rounds = 100
print("Negamax vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle("negamax", "negamax", n_rounds)

## Reactive vs Random

In [ ]:
n_rounds = 100
print("Reactive vs Random")
print(str(n_rounds) + " rounds...")
agent_battle(reactive_agent, random_agent, n_rounds)

## Reactive vs Leftmost

In [ ]:
n_rounds = 100
print("Reactive vs Leftmost")
print(str(n_rounds) + " rounds...")
agent_battle(reactive_agent, leftmost_agent, n_rounds)

## Reactive vs Negamax

In [ ]:
n_rounds = 100
print("Reactive vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle(reactive_agent, "negamax", n_rounds)

## Minimax

In [7]:
n_rounds = 100
print("MiniMax vs Random")
print(str(n_rounds) + " rounds...")
agent_battle(minimax_agent, random_agent, n_rounds)

MiniMax vs Random
100 rounds...
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
--- Results: minimax_agent vs random_agent ---
minimax_agent Win Rate: 1.0
random_agent Win Rate: 0.0
Ties: 0.0
Invalid Plays (Crashes): 0.0


In [12]:
n_rounds = 100
print("MiniMax vs Reactive")
print(str(n_rounds) + " rounds...")
agent_battle(minimax_agent, reactive_agent, n_rounds)

MiniMax vs Reactive
100 rounds...
--- Results: minimax_agent vs reactive_agent ---
minimax_agent Win Rate: 0.95
reactive_agent Win Rate: 0.04
Ties: 0.01
Invalid Plays (Crashes): 0.0


In [14]:
n_rounds = 100
print("MiniMax vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle(minimax_agent, "negamax", n_rounds)

MiniMax vs Negamax
100 rounds...
--- Results: minimax_agent vs negamax ---
minimax_agent Win Rate: 0.5
negamax Win Rate: 0.5
Ties: 0.0
Invalid Plays (Crashes): 0.0


# Custom 2D Board Rendering

In [ ]:
import numpy as np

def render_board(observation, configuration):
    # Convert the flat list to a 2D numpy array (Matrix)
    # This is exactly how your AI will 'see' the board
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    
    print("\n  0 1 2 3 4 5 6") # Column headers
    print(" ---------------")
    for row in grid:
        # Replace numbers with clearer symbols: . (empty), X (P1), O (P2)
        row_str = " ".join(['.' if x == 0 else ('X' if x == 1 else 'O') for x in row])
        print(f"| {row_str} |")
    print(" ---------------")

# Test it
from kaggle_environments import make
env = make("connectx", debug=True)
env.run(["random", "random"])

render_board(env.state[0].observation, env.configuration)